In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\MediBot-An-End-to-End-Medical-Assistance-Chatbot-Powered-by-Generative-AI-and-RAG\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\MediBot-An-End-to-End-Medical-Assistance-Chatbot-Powered-by-Generative-AI-and-RAG'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\ldotm\AppData\Local\Temp\ipykernel_11064\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\anaconda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain.vectorstores import FAISS
docsearch = FAISS.from_documents(text_chunks,embeddings)

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [33]:
retrieved_docs = retriever.invoke("What is Acne?")

In [34]:
retrieved_docs

[Document(id='4121c234-9353-46fa-8619-383e99a56e22', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 39, 'page_label': '40'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='41e155d8-c71d-4e53-b465-d12951fc3382', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 38, 'page_label': '39'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to

In [20]:
from langchain_groq import ChatGroq
llm = ChatGroq()

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "if the answer not realted in medical say i dont know "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [29]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue. Symptoms include unusual height and may occur gradually, often leading to delayed diagnosis. The disorder is relatively rare, affecting approximately 50 out of every one million people.


In [30]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't have enough context to provide a precise answer, but "stats" typically refers to statistics, which involve the collection, analysis, interpretation, and presentation of data. In the provided context, there are some statistical data mentioned, such as the yearly rate for occurrence of sudden unexplained death (SUD) in people less than 35 years of age being less than seven incidents per 100,000.


In [31]:
response = rag_chain.invoke({"input": "What is machine leraning?"})
print(response["answer"])

I don't have enough context to provide an accurate definition of machine learning. However, generally speaking, machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to perform tasks without explicit programming, by learning and improving from experience and data.


In [32]:
response = rag_chain.invoke({"input": "i have a fever waht can do suggest some medicine?"})
print(response["answer"])

To relieve a fever, you can take Acetaminophen (Tylenol) one to two tablets every three to four hours, up to six times per day. Aspirin can also be used to reduce fever, but it's important to note that aspirin can thin the blood and should be avoided if you are at risk of bleeding. If you have a fever, it's important to seek medical attention for a proper diagnosis and treatment.
